# COLMAP Demonstration

A demonstration of using the open-source SFM library [COLMAP](https://colmap.github.io/) to recover cameras and points from the 24 images in the playroom sequence from Assignment 4.

Acknowledgement: This is a modified version of the Colab notebook for [Nerfies Dataset Processing](https://colab.research.google.com/github/google/nerfies/blob/main/notebooks/Nerfies_Capture_Processing.ipynb#scrollTo=saNBv0dY-Eef).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@title Imports

import os
from pathlib import Path      # path and file manipulation
import cv2                    # OpenCV
import numpy as np            # numpy


In [3]:
# @title Configure directories: Edit this cell with folder locations on your google drive

root_dir = Path('/content/drive/My Drive/cs-283-assignments/COLMAP-demo')

# Where the playroom_####.png images will be downloaded
image_dir = root_dir / 'playroom_images'

# Where to save the COLMAP outputs
colmap_dir = root_dir / 'colmap'
colmap_db_path = colmap_dir / 'database.db'
colmap_out_path = colmap_dir / 'sparse'

# Make all of these folders if they do not already exist
colmap_out_path.mkdir(exist_ok=True, parents=True)
image_dir.mkdir(exist_ok=True, parents=True)

print(f"""Directories configured:
  image_dir = {image_dir}
  colmap_dir = {colmap_dir}
  colmap_out_path = {colmap_out_path}
""")

Directories configured:
  image_dir = /content/drive/MyDrive/Courses/CS_283/CS283_2023/Assignments/local/colmap/playroom_images
  colmap_dir = /content/drive/MyDrive/Courses/CS_283/CS283_2023/Assignments/local/colmap/colmap
  colmap_out_path = /content/drive/MyDrive/Courses/CS_283/CS283_2023/Assignments/local/colmap/colmap/sparse



In [4]:
# @title Download playroom sequence to `image-dir`

os.chdir(str(image_dir))

!curl https://codeload.github.com/Harvard-CS283/pset-data/tar.gz/main | \
   tar -xz --strip=3 --wildcards "*/pset4/data/playroom_*.png"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60.2M    0 60.2M    0     0  18.9M      0 --:--:--  0:00:03 --:--:-- 18.9M


In [5]:
#@title Install COLMAP (for reconstruction) and pycolmap (for visualizing output)
!apt-get install colmap
!pip install "git+https://github.com/google/nerfies.git#egg=pycolmap&subdirectory=third_party/pycolmap"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
colmap is already the newest version (3.7-2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
  Cloning https://github.com/google/nerfies.git to /tmp/pip-install-i4t5b4v9/pycolmap_93a4de8bfa25430fb7e726a1d9ac3313
  Running command git clone --filter=blob:none --quiet https://github.com/google/nerfies.git /tmp/pip-install-i4t5b4v9/pycolmap_93a4de8bfa25430fb7e726a1d9ac3313
  Resolved https://github.com/google/nerfies.git to commit 1a38512214cfa14286ef0561992cca9398265e13
  Preparing metadata (setup.py) ... done


In [ ]:
# @title Configure camera settings and extract features

# This is where you specify whether the intrinsic parameters are shared
#   between cameras, and which intrinsic parameters should be estimated. This
#   demo shares intrinsic parameters across all cameras, and it uses the
#   SIMPLE_PINHOLE model, which has no radial distortion, no pixel skew, and the
#   same focal length for the x and y dimensions (i.e., "square pixels with zero skew")
#
#   See: https://colmap.github.io/cameras.html

# Releveant options are:
# --SiftExtraction.upright 1 \                 # turn off SIFT's orientation invariance
# --ImageReader.camera_model SIMPLE_PINHOLE \  # no radial distortion parameters
# --ImageReader.single_camera 1 \              # share intrinsics across all cameras

!colmap feature_extractor \
--SiftExtraction.use_gpu 0 \
--SiftExtraction.upright 1 \
--ImageReader.camera_model SIMPLE_PINHOLE \
--ImageReader.single_camera 1 \
--database_path "{str(colmap_db_path)}" \
--image_path "{str(image_dir)}"


Feature extraction

Processed file [1/24]
  Name:            playroom_00001.png
  Dimensions:      750 x 1000
  Camera:          #1 - SIMPLE_PINHOLE
  Focal Length:    1200.00px
  Features:        2451
Processed file [2/24]
  Name:            playroom_00002.png
  Dimensions:      750 x 1000
  Camera:          #1 - SIMPLE_PINHOLE
  Focal Length:    1200.00px
  Features:        2326
Processed file [3/24]
  Name:            playroom_00004.png
  Dimensions:      750 x 1000
  Camera:          #1 - SIMPLE_PINHOLE
  Focal Length:    1200.00px
  Features:        2545
Processed file [4/24]
  Name:            playroom_00003.png
  Dimensions:      750 x 1000
  Camera:          #1 - SIMPLE_PINHOLE
  Focal Length:    1200.00px
  Features:        2302
Processed file [5/24]
  Name:            playroom_00005.png
  Dimensions:      750 x 1000
  Camera:          #1 - SIMPLE_PINHOLE
  Focal Length:    1200.00px
  Features:        2818
Processed file [6/24]
  Name:            playroom_00006.png
  Dimensi

In [ ]:
# @title Match features

!colmap exhaustive_matcher \
--SiftMatching.use_gpu 0 \
--database_path "{str(colmap_db_path)}"

In [ ]:
# @title Reconstruct

# For description of all of the options, see https://github.com/mwtarnowski/colmap-parameters
# Some of the relevant ones are:
#  --Mapper.ba_refine_principal_point 1       # refine the principal points
#  --Mapper.min_num_matches 32                # minimum viable number of matches between a pair of images

!colmap mapper \
  --Mapper.ba_refine_principal_point 1 \
  --Mapper.filter_max_reproj_error 2 \
  --Mapper.tri_complete_max_reproj_error 2 \
  --Mapper.min_num_matches 32 \
  --database_path "{str(colmap_db_path)}" \
  --image_path "{str(image_dir)}" \
  --output_path "{str(colmap_out_path)}"

In [ ]:
# @title Verify that the reconstruction succeeded

if not colmap_db_path.exists():
  raise RuntimeError(f'The COLMAP DB does not exist, did you run the reconstruction?')
elif not (colmap_dir / 'sparse/0/cameras.bin').exists():
  raise RuntimeError("""
SfM seems to have failed.
""")
else:
  print("Everything looks good!")

In [ ]:
#@title Export reconstructed model to text files (may be helpful to have)
# See https://colmap.github.io/format.html

!colmap model_converter \
    --input_path "{str(colmap_out_path / '0')}" \
    --output_path "{str(colmap_out_path / '0')}" \
    --output_type TXT


In [ ]:
#@title Display camera intrinsic parameters

# Read camera parameters from cameras.txt
with open(str(colmap_out_path / '0/cameras.txt'), 'r') as file:
    lines = file.readlines()

# Last line is of the form CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]
# So get PARAMS[]
params = np.array(lines[-1].strip().split()[4:], dtype=float)

K=np.array([[params[0], 0, params[1]],
   [0, params[0], params[2]],
   [0, 0, 1]])

print(f"Camera intrinsics:\n {K}")

In [ ]:
#@title Use `pycolmap` to access COLMAPs output
import pycolmap
from pycolmap import Quaternion

# Initialize pycolmap scene manager
manager = pycolmap.SceneManager(str(colmap_dir / 'sparse/0'))

# load the model and discard points that do not survive across enough images
manager.load_cameras()
manager.load_images()
manager.load_points3D()
manager.filter_points3D(min_track_len=24)

# Get intrinsic parameters
cameras = manager.cameras     # intrinsic camera parameters (multiple elements if intrinsics were not shared)

# Get 3D points, a Nx3 numpy array
points3D = manager.points3D

# Get the per-image camera information.
#   This returns an ordered dictionary pycolmap.image objects.
#   Each object has attributes:
#   name        : image filename
#   camera_id   : index into cameras, which has the intrinsics
#   q           : camera orientation, a pycolmap.Quarternion object
#   t           : camera center as a list
#   t_vec       : camera center as a numpy array
#   points2D    : detected 2D feature points
#   points3D_ids: index into points3D for each element in points2D
images = manager.images

# Compose lists of camera centers and orientations
camera_centers = []
camera_orientations = []
for key, image in images.items():
    camera_centers.append(image.tvec)
    camera_orientations.append(image.q)

# Convert camera centers to a numpy array
camera_centers=np.array(camera_centers)

In [ ]:
#@title Simple interactive visualization

# To Do: Use the orientation and intrinsics information to draw a square-pyramid for each camera

import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=points3D[:, 0],
    y=points3D[:, 1],
    z=points3D[:, 2],
    mode='markers',
    marker=dict(size=2),
))
fig.add_trace(go.Scatter3d(
    x=camera_centers[:, 0],
    y=camera_centers[:, 1],
    z=camera_centers[:, 2],
    mode='markers',
    marker=dict(size=2),
))
fig.update_traces(showlegend=False)
fig.update_layout(scene_dragmode='orbit')
fig.update_layout(scene=dict(aspectmode='data'))

fig.show()